In [1]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd
import pandas_gbq
from pykrx import stock
from pykrx import bond
import FinanceDataReader as fdr


from time import sleep

import psycopg2 as pg2
from sqlalchemy import create_engine

from datetime import datetime
from datetime import timedelta

import os
import time

import glob
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import storage


from pyarrow import csv
import pyarrow as pa
import pyarrow.parquet as pq

# 경로 변경
os.chdir('/home/shjj08choi4/finance_mlops')


# 서비스 계정 키 JSON 파일 경로
key_path = glob.glob("key_value/*.json")[0]

# Credentials 객체 생성
credentials = service_account.Credentials.from_service_account_file(key_path)

# 빅쿼리 정보
project_id = 'owenchoi-404302'
dataset_id = 'finance_mlops'

# GCP 클라이언트 객체 생성
storage_client = storage.Client(credentials = credentials, 
                         project = credentials.project_id)
bucket_name = 'finance-mlops-proj'     # 서비스 계정 생성한 bucket 이름 입력

# Postgresql 연결
db_connect_info = pd.read_csv('key_value/db_connect_info.csv')
username = db_connect_info['username'][0]
password = db_connect_info['password'][0]
host = db_connect_info['host'][0]
database = db_connect_info['database'][0]
engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:5432/{database}')



In [ ]:
# kor_index_ohlcv
# kor_index_code_fundamental
# kor_index_list_df

In [12]:
# glob.glob('./data_crawler/kor_index_ohlcv/*')
glob.glob('./data_crawler/cleaning/kor_stock_ohlcv/*')

['./data_crawler/cleaning/kor_stock_ohlcv/df_raw_anal_total_2_KOSDAQ_20240107.parquet',
 './data_crawler/cleaning/kor_stock_ohlcv/df_raw_anal_total_2_KOSPI_20240107.parquet',
 './data_crawler/cleaning/kor_stock_ohlcv/df_raw_total_2_KOSDAQ_20240107.parquet',
 './data_crawler/cleaning/kor_stock_ohlcv/df_raw_total_2_KOSPI_20240107.parquet']

In [10]:
for file_nm in ['kor_index_ohlcv','kor_index_code_fundamental','kor_index_list_df']:
    print(file_nm)
#     df = pd.read_csv(f'./data_crawler/{file_nm}/{file_nm}.csv')
#     table_from_pandas = pa.Table.from_pandas(df,preserve_index = False)
#     pq.write_table(table_from_pandas, f'./data_crawler/{file_nm}/{file_nm}.parquet')

    # Google Storage 적재
    source_file_name = f'./data_crawler/{file_nm}/{file_nm}.parquet'   # GCP에 업로드할 파일 절대경로
    destination_blob_name = f'data_crawler/{file_nm}/{file_nm}.parquet'    # 업로드할 파일을 GCP에 저장할 때의 이름
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)
    

kor_index_ohlcv
kor_index_code_fundamental
kor_index_list_df


In [5]:
glob.glob('./data_crawler/kor_index_code_fundamental/*')

['./data_crawler/kor_index_code_fundamental/kor_index_code_fundamental.csv']

In [6]:
glob.glob('./data_crawler/kor_index_list_df/*')

['./data_crawler/kor_index_list_df/kor_index_list_df.csv']

In [ ]:
/home/shjj08choi4/finance_mlops/data_crawler/cleaning/kor_stock_ohlcv

### ghkrdls

In [13]:
t = pq.read_table('./data_crawler/cleaning/kor_stock_ohlcv/df_raw_total_2_KOSPI_20240107.parquet')
df = t.to_pandas()

In [17]:
df[df['ticker'] == '005930'].sort_values(by = 'date')

,date,open,high,low,close,volume,price_change_percentage,ticker,corp_name,market,...,MACD_Signal,변화량,상승폭,하락폭,AU,AD,RSI,MA5-20,MA20-60,MA60-120
100580,2023-01-02 00:00:00+00:00,55500,56100,55200,55500.0,10031448,0.361664,005930,삼성전자,KOSPI,...,-51.423271,300.0,300.0,0.0,2524.744891,2792.099375,47.485779,-7343.5,-1257.333333,-317.083333
100730,2023-01-03 00:00:00+00:00,55400,56000,54500,55400.0,13547030,-0.180180,005930,삼성전자,KOSPI,...,56.758129,200.0,200.0,0.0,2299.415740,2427.913626,48.640904,-6977.5,1627.500000,-2060.583333
100685,2023-01-04 00:00:00+00:00,55700,58000,55600,57800.0,20188071,4.332130,005930,삼성전자,KOSPI,...,-1145.081517,1200.0,1200.0,0.0,2258.424768,2399.666463,48.483910,-10405.0,4880.500000,-1367.083333
100716,2023-01-05 00:00:00+00:00,58200,58800,57600,58200.0,15682826,0.692042,005930,삼성전자,KOSPI,...,-59.694388,12900.0,12900.0,0.0,3210.535134,3358.790765,48.871607,-15852.5,166.000000,-226.500000
100547,2023-01-06 00:00:00+00:00,58300,59400,57900,59000.0,17334989,1.374570,005930,삼성전자,KOSPI,...,-2176.220957,7100.0,7100.0,0.0,2610.883430,2665.237807,49.484902,-10303.5,3284.166667,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100709,2023-12-28 00:00:00+00:00,77700,78500,77500,78500.0,17797536,0.641026,005930,삼성전자,KOSPI,...,2014.186755,700.0,700.0,0.0,2514.553248,1469.548906,63.114678,11184.5,593.166667,765.333333
100725,2024-01-02 00:00:00+00:00,78200,79800,78200,79600.0,17142847,1.401274,005930,삼성전자,KOSPI,...,2826.684116,1000.0,1000.0,0.0,2464.276013,1629.853306,60.190478,10050.0,2666.666667,1540.083333
100741,2024-01-03 00:00:00+00:00,78500,78800,77000,77000.0,21753644,-3.266332,005930,삼성전자,KOSPI,...,3114.837090,-800.0,0.0,800.0,2118.356444,1095.963762,65.903715,9915.0,6018.666667,-104.333333
100773,2024-01-04 00:00:00+00:00,76100,77300,76100,76600.0,15324439,-0.519481,005930,삼성전자,KOSPI,...,1509.125736,5700.0,5700.0,0.0,2382.622469,1481.168962,61.665401,9770.0,-970.833333,169.166667
